In [1]:
from qreduce.cs_vqe import cs_vqe_model
from qreduce.utils import * 
from itertools import combinations
import json
import numpy as np

In [28]:
with open('data/model_data.json', 'r') as infile:
        mol_data = json.load(infile)
mol_data.keys()

dict_keys(['H2O_STO-3G_SINGLET', 'LiH_STO-3G_SINGLET', 'HF_STO-3G_SINGLET', 'CH2_STO-3G_SINGLET', 'BeH2_STO-3G_SINGLET', 'HeH+_6-311G_SINGLET', 'BeH+_STO-3G_SINGLET', 'CH+_STO-3G_SINGLET', 'B+_STO-3G_SINGLET', 'OH+_STO-3G_SINGLET', 'OH-_STO-3G_SINGLET', 'O_STO-3G_SINGLET', 'Be_STO-3G_SINGLET', 'H4_STO-3G_SINGLET', 'B_STO-3G_DOUBLET', 'NH_STO-3G_SINGLET', 'BH_STO-3G_SINGLET', 'NH2+_STO-3G_SINGLET', 'BH2+_STO-3G_SINGLET', 'CH2_STO-3G_TRIPLET', 'C_STO-3G_TRIPLET'])

In [30]:
mol=mol_data['O_STO-3G_SINGLET']
ham=mol['ham']
n_q=mol['num_qubits']
set_nc = mol['terms_noncon']
cs_vqe_mol = cs_vqe_model(ham, n_q, noncontextual_set = set_nc)

In [31]:
print("Generators:", cs_vqe_mol.generators)
print("Clique reps:", cs_vqe_mol.cliquereps)
match=mol['X_index']==cs_vqe_mol.anti_clique_qubit
print('Anti-clique qubit match?', match, '\n')

print("G assignment:", cs_vqe_mol.nu)#, ' | original:', np.array(list(mol['G'].values()), dtype=int))
print("r vector:", cs_vqe_mol.r)
print("Noncon nrg:", cs_vqe_mol.ngs_energy)
print('NGS matches original?', abs(cs_vqe_mol.ngs_energy-mol['noncon'])<1e-13, '\n')

print("Stabilizers:", cs_vqe_mol.stabilizers)
print("Stab qubits:", cs_vqe_mol.stab_qubits)

Generators: ['ZZZZI', 'ZIZZI', 'ZZIZI', 'ZZZII']
Clique reps: ['ZZZZZ', 'ZZZZX']
Anti-clique qubit match? True 

G assignment: [-1  1  1  1]
r vector: [1.00000000e+00 4.19232508e-10]
Noncon nrg: -73.75670578813755
NGS matches original? True 

Stabilizers: {'ZIIII': 1, 'IZIII': 1, 'IIZII': 1, 'IIIZI': 1, 'IIIIZ': -1.0}
Stab qubits: [0, 1, 2, 3, 4]


In [32]:
cs_vqe_mol.ngs_energy,mol['noncon']

(-73.75670578813755, -73.75670578813754)

In [33]:
num_sim_q = mol['chem_acc_num_q']
exact = exact_gs_energy(cs_vqe_mol.ham)
print('Exact energy:',exact)
print('Noncon error:', cs_vqe_mol.ngs_energy-exact)
print(f'Target_error for {num_sim_q} qubits:', exact_gs_energy(mol['ham_reduced'][num_sim_q])-exact)

Exact energy: -73.80415023325594
Noncon error: 0.04744444511838708
Target_error for 1 qubits: 1.4210854715202004e-14


In [39]:
for comb in combinations(range(n_q), num_sim_q):
    comb = list(comb)# mol['order'][0:mol['chem_acc_num_q']] #=list(comb)
    ham_cs = cs_vqe_mol.contextual_subspace_hamiltonian_2(comb)
    #print(ham_cs)
    cs_energy = exact_gs_energy(ham_cs)
    in_range = cs_energy<cs_vqe_mol.ngs_energy and cs_energy>exact
    print(f'error for qubits {comb}:', cs_energy-exact, '| In expected range?', in_range)
    #print(ham_cs)

error for qubits [0]: 0.04744444511837287 | In expected range? True
error for qubits [1]: 0.04744444511838708 | In expected range? False
error for qubits [2]: 1.4210854715202004e-14 | In expected range? True
error for qubits [3]: 0.04744444511840129 | In expected range? False
error for qubits [4]: 0.04744444511840129 | In expected range? False
